# BS pricing strategies

In [2]:
# The Black-Scholes prices
def bs_put(t, S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + 1/2 * sigma**2) * (T-t)) / sigma / np.sqrt(T-t)
    d2 = (np.log(S0/K) + (r - 1/2 * sigma**2) * (T-t)) / sigma / np.sqrt(T-t)
    price = K * np.exp(-r * (T-t)) * norm.cdf(-d2) - S0 * norm.cdf(-d1)

def bs_call(t, S0, K, r, sigma, T):
    d1 = (np.log(S0/K) + (r + 1/2 * sigma**2) * (T-t)) / sigma / np.sqrt(T-t)
    d2 = (np.log(S0/K) + (r - 1/2 * sigma**2) * (T-t)) / sigma / np.sqrt(T-t)
    price = S0 * norm.cdf(d1) - K * np.exp(-r * (T-t)) * norm.cdf(d2)
    return price

def d1(S0, K, r, sigma, T):
    return (np.log(S0/K) + (r + sigma**2 / 2) * T)/(sigma * np.sqrt(T))
 
def d2(S0, K, r, sigma, T):
    return (np.log(S0 / K) + (r - sigma**2 / 2) * T) / (sigma * np.sqrt(T))


In [20]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
%matplotlib inline

from numpy.random import standard_normal, seed

import scipy.stats as stats
from scipy.stats import norm

import sys

sys.path.append("..")

import datetime 
import time
import bspline
import bspline.splinelab as splinelab

# DiscreteBlackScholes class

In [21]:
# Note that we use splines to approximate the underlying random-walking function as piecewise polynomial 
class DiscreteBlackScholes:

    # Note that if you have large-scale date set, initialization might take sometime to implement
    # So do not duplicately initialize objects with same data!
    def __init__(self,
                 s0,
                 strike,
                 vol,
                 T,
                 r,
                 mu,
                 numSteps,
                 numPaths):
        """
        :param s0: initial price of the underlying
        :param strike: option strike
        :param vol: volatility
        :param T: time to maturity, in years
        :param r: risk-free rate,
        :param mu: real drift, asset drift
        :param numSteps: number of time steps
        :param numPaths: number of Monte Carlo paths
        """
        self.s0 = s0
        self.strike = strike
        self.vol = vol
        self.T = T
        self.r = r
        self.mu = mu
        self.numSteps = numSteps
        self.numPaths = numPaths

        self.dt = self.T / self.numSteps  # time step
        self.gamma = np.exp(-r * self.dt)  # discount factor for one time step, i.e. gamma in the QLBS paper

        self.sVals = np.zeros((self.numPaths, self.numSteps + 1), 'float')  # matrix of stock values

        # initialize half of the paths with stock price values ranging from 0.5 to 1.5 of s0
        # the other half of the paths start with s0
        half_paths = int(numPaths / 2)

        if False:
            # Grau (2010) "Applications of Least-Squares Regressions to Pricing and Hedging of Financial Derivatives"
            self.sVals[:, 0] = (np.hstack((np.linspace(0.5 * s0, 1.5 * s0, half_paths),
                                           s0 * np.ones(half_paths, 'float')))).T

        self.sVals[:, 0] = s0 * np.ones(numPaths, 'float')
        self.optionVals = np.zeros((self.numPaths, self.numSteps + 1), 'float')  # matrix of option values
        self.intrinsicVals = np.zeros((self.numPaths, self.numSteps + 1), 'float')

        self.bVals = np.zeros((self.numPaths, self.numSteps + 1), 'float')  # matrix of cash position values
        self.opt_hedge = np.zeros((self.numPaths, self.numSteps + 1),
                              'float')  # matrix of optimal hedges calculated from cross-sectional information F_t
        self.X = None
        self.data = None  # matrix of features, i.e. self.X as sum of basis functions
        self.delta_S_hat = None

        # coef = 1.0/(2 * gamma * risk_lambda)
        # override it by zero to have pure risk hedge
        self.coef = 0.

    # generate path for the MCMC estimation of parameters
    def gen_paths(self):
        """
        A simplest path generator
        """
        np.random.seed(79)
        Z = np.random.normal(0, 1, size=(self.numSteps + 1, self.numPaths)).T
        for t in range(0, self.numSteps):
            self.sVals[:, t + 1] = self.sVals[:, t] * np.exp((self.mu - 0.5 * self.vol**2) * self.dt + (self.vol * np.sqrt(self.dt) * Z[:, t + 1]))
        
        print(self.sVals)
        
        # compute the delta_S based on quadra-linear BS
        delta_S = self.sVals[:, 1:] - np.exp(self.r * self.dt) * self.sVals[:, :self.numSteps]
        self.delta_S_hat = np.apply_along_axis(lambda x: x - np.mean(x), axis=0, arr=delta_S)

        # we compute the state variable delta_t based on conventions
        self.X = - (self.mu - 0.5 * self.vol ** 2) * np.arange(self.numSteps + 1) * self.dt + np.log(self.sVals)

        X_min = np.min(np.min(self.X))
        X_max = np.max(np.max(self.X))
        
        print('X.shape = ', self.X.shape)
        print('X_min, X_max = ', X_min, X_max)

        p = 4    # order of the piecewise polynomial B-spline function
        ncolloc = 12 # note that we should have ncolloc>=p+1 to estimate the parameter of B-spline functions
        tau = np.linspace(X_min, X_max, ncolloc)  # use linear space to generate sites where we want to intrpolate
        
        # k is a knot vector that adds endpoints repeats as appropriate for a spline of order p
        k = splinelab.aptknt(tau, p)   
        basis = bspline.Bspline(k, p)
        num_basis = ncolloc
        self.data = np.zeros((self.numSteps + 1, self.numPaths, num_basis))
        
        print('num_basis = ', num_basis)
        print('dim self.data = ', self.data.shape)
        
        # fill the spline by expanding in finite-dimentional space
        # in NN, the basis is the NN it self
        t_0 = time.time()
        for ix in np.arange(self.numSteps + 1):
            x = self.X[:, ix]
            self.data[ix, :, :] = np.array([basis(el) for el in x])
        t_end = time.time()
        print('\nTime Cost of basis expansion:', t_end - t_0, 'seconds')
        
    def function_A_vec(self, t, reg_param=1e-3):
        # compute the Matrix A
        X_mat = self.data[t, :, :]
        num_basis_funcs = X_mat.shape[1]
        this_dS = self.delta_S_hat[:, t]
        hat_dS2 = (this_dS ** 2).reshape(-1, 1)
        A_mat = np.dot(X_mat.T, X_mat * hat_dS2) + reg_param * np.eye(num_basis_funcs)
        return A_mat
    
    def function_B_vec(self, t, Pi_hat):
        # compute the vector B
        tmp = Pi_hat * self.delta_S_hat[:, t] + self.coef * (np.exp((self.mu - self.r) * self.dt)) * self.sVals[:, t]
        X_mat = self.data[t, :, :]  # matrix of dimension N_MC x num_basis

        B_vec = np.dot(X_mat.T, tmp)
        return B_vec
    
    def seed_intrinsic(self, strike=None, cp='P'):
        # initialization of option value and intrinsic value on each node
        if strike is not None:
            self.strike = strike

        if cp == 'P':
            self.optionVals = np.maximum(self.strike - self.sVals[:, -1], 0).copy()
            
        elif cp == 'C':
            self.optionVals = np.maximum(self.sVals[:, -1] - self.strike, 0).copy()
            self.intrinsicVals = np.maximum(self.sVals - self.strike, 0).copy()
        
        else:
            raise Exception('Invalid parameter: %s'% cp)
            
        self.bVals[:, -1] = self.intrinsicVals[:, -1]
        
    def roll_backward(self):
        # roll the price and optimal hedge back in time starting from maturity
        for t in range(self.numSteps - 1, -1, -1):
            piNext = self.bVals[:, t+1] + self.opt_hedge[:, t+1] * self.sVals[:, t+1]
            pi_hat = piNext - np.mean(piNext)
            
            A_mat = self.function_A_vec(t)
            B_vec = self.function_B_vec(t, pi_hat)
            phi = np.dot(np.linalg.inv(A_mat), B_vec)
            self.opt_hedge[:, t] = np.dot(self.data[t, :, :], phi)
            self.bVals[:,t] = np.exp(-self.r * self.dt) * (self.bVals[:, t+1] + (self.opt_hedge[:, t+1] - self.opt_hedge[:, t]) * self.sVals[:, t+1])
      
        # calculate the initial portfolio value
        initPortfolioVal = self.bVals[:, 0] + self.opt_hedge[:, 0] * self.sVals[:, 0]

        # use only the second half of the paths generated with paths starting from S0
        optionVal = np.mean(initPortfolioVal)
        optionValVar = np.std(initPortfolioVal)
        delta = np.mean(self.opt_hedge[:, 0])

        return optionVal, delta, optionValVar

In [22]:
np.random.seed(42)
strike_k = 95
test_vol = 0.2
test_mu = 0.03
dt = 0.01
rfr = 0.05
num_paths = 100
num_periods = 252

hMC = DiscreteBlackScholes(100, strike_k, test_vol, 1., rfr, test_mu, num_periods, num_paths)
hMC.gen_paths()

t = hMC.numSteps - 1
piNext = hMC.bVals[:, t+1] + 0.1 * hMC.sVals[:, t+1]
pi_hat = piNext - np.mean(piNext)

A_mat = hMC.function_A_vec(t)
B_vec = hMC.function_B_vec(t, pi_hat)
phi = np.dot(np.linalg.inv(A_mat), B_vec)
opt_hedge = np.dot(hMC.data[t, :, :], phi)

# plot the results
# fig = plt.figure(figsize=(12,4))
# ax1 = fig.add_subplot(121)

# ax1.scatter(hMC.sVals[:,t], pi_hat)
# ax1.set_title(r'Expected $\Pi_0$ vs. $S_t$')
# ax1.set_xlabel(r'$S_t$')
# ax1.set_ylabel(r'$\Pi_0$')

[[100.         100.25654696  98.92161574 ... 101.05694817  99.80931104
   98.8343727 ]
 [100.          99.72442566  99.566852   ... 100.11266523 100.08106833
  100.12055704]
 [100.         101.1010862  100.10173414 ... 176.95880218 179.07807085
  177.67544136]
 ...
 [100.         102.85987929 101.82475808 ... 111.12808351 111.2574908
  110.92616969]
 [100.         100.79686929 101.83418939 ... 100.84915795 101.35445388
  100.97797509]
 [100.         100.01431849 101.57283967 ...  75.1422716   74.15795775
   74.20894163]]
X.shape =  (100, 253)
X_min, X_max =  4.155663632371381 5.178136537043684
num_basis =  12
dim self.data =  (253, 100, 12)

Time Cost of basis expansion: 2.5406548976898193 seconds
